# Initiliation

In [1]:
!pip install transformers datasets --quiet

     |████████████████████████████████| 3.4 MB 8.4 MB/s 
     |████████████████████████████████| 306 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 66.8 MB/s 
     |████████████████████████████████| 61 kB 483 kB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 1.1 MB 51.9 MB/s 
     |████████████████████████████████| 132 kB 67.8 MB/s 
     |████████████████████████████████| 243 kB 76.1 MB/s 
     |████████████████████████████████| 271 kB 55.5 MB/s 
     |████████████████████████████████| 192 kB 61.3 MB/s 
     |████████████████████████████████| 160 kB 50.3 MB/s 


In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/URL'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW myPers/00_Datasets/URL


In [5]:
dfPJ = pd.read_csv('myPers_AGR_URL.csv', sep=",", error_bad_lines=False)
dfPJ

,text,label
0,likes the sound of thunder,0
1,is so sleepy its not even funny thats ...,0
2,is sore and wants the knot of muscles ...,0
3,likes how the day sounds in this new s...,0
4,is home 3,0
...,...,...
9912,little things give you away,1
9913,is wishing it was saturday,1
9914,is studying hard for the gre,1
9915,snipers get more head,0


In [6]:
df_clean= []
for i,row in dfPJ.iterrows():
  df_clean.append({
      'text': str(row['text']),
      'label': int(row['label']),
  })

dfPJ = pd.DataFrame(df_clean)
dfPJ

,text,label
0,likes the sound of thunder,0
1,is so sleepy its not even funny thats ...,0
2,is sore and wants the knot of muscles ...,0
3,likes how the day sounds in this new s...,0
4,is home 3,0
...,...,...
9912,little things give you away,1
9913,is wishing it was saturday,1
9914,is studying hard for the gre,1
9915,snipers get more head,0


# Model Training

In [7]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [8]:
train, test = train_test_split(dfPJ, test_size=0.2, random_state=0, stratify=dfPJ.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_AGR", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [9]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 9.1 MB/s 
     |████████████████████████████████| 80 kB 10.9 MB/s 
     |████████████████████████████████| 209 kB 72.6 MB/s 
     |████████████████████████████████| 75 kB 5.6 MB/s 
     |████████████████████████████████| 112 kB 66.5 MB/s 
     |████████████████████████████████| 49 kB 7.6 MB/s 
     |████████████████████████████████| 149 kB 75.4 MB/s 


In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [11]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

In [12]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_myPers_AGR"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=8, direction="maximize",study_name=study_name, storage=storage_name,load_if_exists=True )

study = optuna.create_study()

[I 2021-12-25 15:55:40,162] Using an existing study with name 'SIMCSE_BERT_myPers_AGR' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_myPers_AGR' instead of creating a new one.


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-bert-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weigh

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.685300,0.688053,0.555948,0.554178,0.554181,0.554175
2,0.691100,0.846266,0.560988,0.560218,0.560353,0.560563
3,0.830200,1.565335,0.579133,0.578395,0.578507,0.578779
4,0.619300,2.134844,0.586694,0.579916,0.583782,0.581025


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       930
           1       0.53      1.00      0.69      1054

    accuracy                           0.53      1984
   macro avg       0.27      0.50      0.35      1984
weighted avg       0.28      0.53      0.37      1984

[[   0  930]
 [   0 1054]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Trial 51 pruned. 


[I 2021-12-25 19:41:16,909] A new study created in memory with name: no-name-4e3d942c-7d72-427d-9ab1-b4c95500d597


A new study created in memory with name: no-name-4e3d942c-7d72-427d-9ab1-b4c95500d597


In [13]:
storage_name

'sqlite:///SIMCSE_BERT_myPers_AGR.db'

In [14]:
study_name

'SIMCSE_BERT_myPers_AGR'

In [15]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-25 19:41:17,004] Using an existing study with name 'SIMCSE_BERT_myPers_AGR' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_myPers_AGR' instead of creating a new one.


In [16]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.493657,1.605051e-06,3.088936,16,1.274813e-09,COMPLETE
1,1,0.319149,1.075785e-01,2.258434,4,9.147818e-02,COMPLETE
2,2,0.346939,5.506321e-05,3.049885,8,5.831691e-04,COMPLETE
3,3,0.459720,2.001532e-07,5.299271,16,6.342396e-11,COMPLETE
4,4,0.346939,2.127109e-02,3.430230,8,2.079299e-11,COMPLETE
5,5,0.346939,8.104737e-03,1.594893,2,6.557118e-02,COMPLETE
6,6,0.570077,3.033881e-06,6.331414,4,1.716952e-03,COMPLETE
7,7,0.319149,5.584370e-02,2.715283,4,3.289601e-11,PRUNED
8,8,0.346939,2.522861e-04,1.764025,4,8.964743e-11,COMPLETE
9,9,0.346939,5.658322e-04,4.202345,8,4.178087e-04,PRUNED


In [17]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [18]:
best_run

BestRun(run_id='12', objective=0.6002162006600213, hyperparameters={'learning_rate': 1.1303968708302916e-05, 'num_train_epochs': 7.7171833770773, 'per_device_train_batch_size': 2, 'weight_decay': 1.3208253773473318e-07})

In [19]:
optuna.visualization.plot_intermediate_values(study)

In [20]:
optuna.visualization.plot_parallel_coordinate(study)

In [21]:
optuna.visualization.plot_optimization_history(study)

In [22]:
optuna.visualization.plot_contour(study)

In [23]:
optuna.visualization.plot_slice(study)

In [24]:
optuna.visualization.plot_edf(study)